In [1]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import eval_metrics as em
import wandb
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.metrics import classification_report

### Configurations

In [2]:
run = wandb.init(
    project = "teamlab_deepfake",
    config={
        "learning_rate": 0.001,
        "model": "LSTM",    #NOTE: set manually
        "dataset": "ASVSpoof19_LA_original",    #NOTE: set manually
        "feature": "PITCH_HNR_scaled",   #NOTE: set manually
        "epochs": 1,
        "batch_size": 32,
        "lstm_input_dim": 2,
        "lstm_hidden_dim": 64,
        "bidirectional": False,
        "lstm_n_layers":1,
        "dropout_rate": 0,
        "ffn_input_dim": 11,
        "ffn_hidden_dim": 64,
        "loss_function": "weighted_CE",     #NOTE: set manually
    },
    name = "small_test",     #NOTE: set manually
    notes = None,
    tags = ["multi_feature","HNR", "PITCH", "scaling"],
)

config = run.config

wandb: Currently logged in as: qianyue (qianyue-university-of-stuttgart) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
PITCH_COLUMN = 'PITCH'
HNR_COLUMN = 'HNR'
JITTER_COLUMN = 'JITTER'
SHIMMER_COLUMN = 'SHIMMER'
AUDIO_ID_COLUMN = 'AUDIO_ID'
LABEL_COLUMN = 'LABEL'
ATTACK_COLOMN = 'ATTACK_TYPE'        
                           
NAN_REPLACEMENT_VALUE = 0.0  
PADDING_VALUE = 0.0         
LABEL_BONAFIDE = 1
LABEL_SPOOF = 0

train_features_path = '/home/users1/liqe/TeamLab_phonetics/prosody_features_train_scaled.pkl'
dev_features_path = '/home/users1/liqe/TeamLab_phonetics/prosody_features_dev_scaled.pkl'

df_train = pd.read_pickle(train_features_path)
df_dev = pd.read_pickle(dev_features_path)

### Training Data Oversampling

>NOTE: training audio & labels are matched, dev are not (Solved: excessive rows are deleted beforehand)

In [4]:
X = df_train.drop('LABEL', axis=1)
y = df_train['LABEL']

over = RandomOverSampler(random_state=42)
X_resampled_np, y_resampled_np = over.fit_resample(X, y) 

X_resampled_df = pd.DataFrame(X_resampled_np, columns=X.columns)
y_resampled_series = pd.Series(y_resampled_np, name=y.name)

print("\nResampled X (DataFrame) head:")
print(X_resampled_df.head())
print("\nResampled y (Series) head:")
print(y_resampled_series.head())
print("\nResampled class distribution (from y_resampled_series):")
print(Counter(y_resampled_series))

train_resampled = pd.concat([X_resampled_df, y_resampled_series], axis=1)

print("\nCombined Resampled DataFrame head:")
print(train_resampled.head())
print("\nCombined Resampled DataFrame info:")
train_resampled.info()
print("\nCombined Resampled DataFrame class distribution:")
print(Counter(train_resampled['LABEL'])) # Verify target column in the new DataFrame


Resampled X (DataFrame) head:
       AUDIO_ID ATTACK_TYPE  \
0  LA_T_1000137         A04   
1  LA_T_1000406           -   
2  LA_T_1000648         A01   
3  LA_T_1000824         A04   
4  LA_T_1001074         A03   

                                               PITCH  \
0  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
1  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
2  [nan, nan, nan, nan, nan, 0.35835335634967486,...   
3  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
4  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                                 HNR  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7987432090825...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                              JITTER  \
0  [0.30073947, 0.23022015, 0.26707897, 0.2538646...

### Scaling
>note: need custom function to scale if not padded (solved: scaled data saved)

### Padding and Data Loader

In [5]:
# NOTE: CHANGE THE NAME LATER
class PitchHNRDataset(Dataset):
    def __init__(self, dataframe, pitch_col, hnr_col, jitter_col, shimmer_col, audio_id_col, label_col, nan_replacement=NAN_REPLACEMENT_VALUE):
        
        self.processed_features = []
        self.labels = []
        self.global_features = []
        
        print(f"Attempting to process {len(dataframe)} entries from DataFrame")
        found_count = 0
        # Iterate through the DataFrame, process and pad the features
        for index, row in dataframe.iterrows():  
            if not np.isnan(row[label_col]):
                self.labels.append(row[label_col])

                pitch_sequence_raw = row[pitch_col]
                processed_pitch = np.nan_to_num(pitch_sequence_raw, nan=nan_replacement)
                
                hnr_sequence_raw = row[hnr_col]
                processed_hnr = np.nan_to_num(hnr_sequence_raw, nan=nan_replacement)

                ### NOTE:need to pad the two sequences to the same length
                max_length = max(len(processed_pitch), len(processed_hnr))
                if len(processed_pitch) > len(processed_hnr):
                    padding = np.zeros(max_length - len(processed_hnr), dtype=processed_hnr.dtype)
                    processed_hnr = np.concatenate((processed_hnr, padding))
                else:
                    padding = np.zeros(max_length - len(processed_pitch), dtype=processed_pitch.dtype)
                    processed_pitch = np.concatenate((processed_pitch, padding))

                combined_features = np.stack((processed_pitch, processed_hnr), axis=-1) 
                self.processed_features.append(torch.tensor(combined_features, dtype=torch.float32))

                # process and combine jitter and shimmer to one sequence
                processed_jitter = np.nan_to_num(row[jitter_col], nan=nan_replacement)
                processed_shimmer = np.nan_to_num(row[shimmer_col], nan=nan_replacement)
                jitter_shimmer = np.concatenate((processed_jitter, processed_shimmer))
                self.global_features.append(torch.tensor(jitter_shimmer, dtype=torch.float32))
                
                found_count += 1
        
        if not self.processed_features:
            raise ValueError("No samples were successfully matched and processed. Check your AUDIO_IDs and label file.")

        self.labels = torch.tensor(self.labels, dtype=torch.long) # Assuming labels are integers for classification
        print(f"Successfully processed {found_count} samples out of {len(dataframe)} DataFrame entries.")


    def __len__(self):
        """Returns the total number of matched samples in the dataset."""
        return len(self.processed_features)

    def __getitem__(self, idx):
        """
        Returns one sample from the dataset: a preprocessed pitch sequence and its label.
        """
        feature_sequence = self.processed_features[idx]
        label = self.labels[idx]
        global_feature = self.global_features[idx]
        return feature_sequence, label, global_feature

In [6]:
# --- Custom Collate Function for Dynamic Padding  ---
def collate_fn(batch, padding_value=PADDING_VALUE):
    """
    Pads sequences within a batch to the same length.
    """
    sequences = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    global_features = [item[2] for item in batch]
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=padding_value)
    labels = torch.stack(labels)
    if padded_sequences.ndim == 2:
        padded_sequences = padded_sequences.unsqueeze(2)
    global_features = torch.stack(global_features)
    return padded_sequences, lengths, global_features, labels

In [7]:
pitch_dataset_train = PitchHNRDataset(dataframe=train_resampled,   # NOTE: oversampled (change upon need)
                                     pitch_col=PITCH_COLUMN,
                                     hnr_col=HNR_COLUMN,
                                     jitter_col=JITTER_COLUMN,
                                     shimmer_col=SHIMMER_COLUMN,
                                     audio_id_col=AUDIO_ID_COLUMN,
                                     label_col=LABEL_COLUMN,
                                     nan_replacement=NAN_REPLACEMENT_VALUE)

train_dataloader = DataLoader(
    pitch_dataset_train, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn
)

pitch_dataset_dev = PitchHNRDataset(dataframe=df_dev,   # NOTE: oversampled (change upon need)
                                     pitch_col=PITCH_COLUMN,
                                     hnr_col=HNR_COLUMN,
                                     jitter_col=JITTER_COLUMN,
                                     shimmer_col=SHIMMER_COLUMN,
                                     audio_id_col=AUDIO_ID_COLUMN,
                                     label_col=LABEL_COLUMN,
                                     nan_replacement=NAN_REPLACEMENT_VALUE)

dev_dataloader = DataLoader(
    pitch_dataset_dev, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn
)

## For inspection
for i, batch_data in enumerate(train_dataloader):
    # batch_data is a tuple
    batch_sequences, batch_lengths, batch_global, batch_labels = batch_data
    print(f"\n--- Batch {i+1} ---")
    print(f"  Padded Sequences Shape: {batch_sequences.shape}")
    print(f"  Original Lengths (first 5): {batch_lengths[:5]}")
    print(f"  Global Shape: {batch_global.shape}")
    print(f"  Labels (first 5): {batch_labels[:5]}")
    

    if i == 0: # Break after the first batch for inspection
        break


Attempting to process 45598 entries from DataFrame


Successfully processed 45598 samples out of 45598 DataFrame entries.
Attempting to process 24844 entries from DataFrame
Successfully processed 24844 samples out of 24844 DataFrame entries.

--- Batch 1 ---
  Padded Sequences Shape: torch.Size([32, 682, 2])
  Original Lengths (first 5): tensor([292, 377, 245, 347, 208])
  Global Shape: torch.Size([32, 11])
  Labels (first 5): tensor([1, 0, 1, 1, 0])


### Finding the weight (for weighted cross entropy)

is there different ways calculating weitghs?

In [8]:

labels = train_resampled['LABEL']   # NOTE:w/ oversampling
#labels = df_train['LABEL']   # NOTE:w/o oversampling
total = len(labels)
count_bonafide = labels.value_counts().get(LABEL_BONAFIDE, 0)
count_spoof =  total - count_bonafide
weight_bonafide = total / (count_bonafide * 2)
weight_spoof = total / (count_spoof * 2)

### Classifier

In [9]:
class SpoofClassifier(nn.Module):
    def __init__(self, lstm_input_dim, lstm_hidden_dim, lstm_n_layers,
                 bidirectional, dropout, ffn_input_dim, ffn_hidden_dim, output_dim,):

        super().__init__()
        
        # self.lstm_input_dim = lstm_input_dim
        # self.lstm_hidden_dim = lstm_hidden_dim
        # self.lstm_output_dim = lstm_output_dim
        # self.lstm_n_layers = lstm_n_layers
        # self.bidirectional = bidirectional
        # self.ffn_input_dim = ffn_input_dim
        # self.ffn_hidden_dim = ffn_hidden_dim
        
        # 1. LSTM Layer
        self.lstm = nn.LSTM(lstm_input_dim, 
                            lstm_hidden_dim, 
                            num_layers=lstm_n_layers, 
                            bidirectional=bidirectional, 
                            dropout=dropout if lstm_n_layers > 1 else 0,
                            batch_first=True) # Input/output tensors are (batch, seq, feature)
        # BN layer for stabalization
        self.bn_lstm = nn.BatchNorm1d(lstm_hidden_dim)
        
        # 2. ffn layer
        self.ffn = nn.Sequential(
            nn.Linear(ffn_input_dim, ffn_hidden_dim),
            nn.BatchNorm1d(ffn_hidden_dim),    # BN layer for stabalization
            nn.ReLU(),
            nn.Dropout(dropout))

        # self.ffn_linear = nn.Linear(self.ffn_input_dim, self.ffn_hidden_dim)
        # self.ffn_relu = nn.ReLU()
        # self.ffn_dropout = nn.Dropout(dropout) # Use the dropout value passed to __init__
        
        
        # Explicitly initialize the linear layer's weights/biases
        # nn.init.kaiming_uniform_(self.ffn[0].weight, nonlinearity='relu')
        # if self.ffn[0].bias is not None:
        #     nn.init.zeros_(self.ffn[0].bias)
        
        # 3. Fully Connected Layer (Linear Layer)
        self.fc = nn.Linear((lstm_hidden_dim * 2 if bidirectional else lstm_hidden_dim) + ffn_hidden_dim, output_dim)
        
        # 4. Dropout Layer (for regularization on the output of LSTM)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, sequences, sequence_lengths, global_features):
      
        # 1. Pack sequence
        ### Compute actual data and ignore the padded values
        packed_input = rnn_utils.pack_padded_sequence(sequences, sequence_lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # 2. Pass packed sequence through LSTM
        ### packed_output: Hidden states for every time step.
        ### hidden: The final hidden state (summary) of the entire sequence.
        ### cell: The final cell state (long-term memory) of the entire sequence.
        packed_output, (hidden, cell) = self.lstm(packed_input)
        
        # 3. Concatenate the final forward and backward hidden states (if bidirectional)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        hidden = self.bn_lstm(hidden)

        #print(f"DEBUG: lstm_output - Min:{hidden.min():.2e}, Max:{hidden.max():.2e}, Mean:{hidden.mean():.2e}, Has_NaN:{torch.isnan(hidden).any()}")

        # 4. Pass global features (jitter and shimmer) through the FFN
        ffn_output = self.ffn(global_features)

        # print(f"DEBUG: ffn_output - Min:{ffn_output.min():.2e}, Max:{ffn_output.max():.2e}, Mean:{ffn_output.mean():.2e}, Has_NaN:{torch.isnan(ffn_output).any()}")

        #print(f"DEBUG: global_features BEFORE FFN_Linear (Sample 0): {global_features[0].detach().cpu().numpy()}")
        

        # 5. Concatenate the outputs from lstm and fnn
        combined_features = torch.cat((hidden,ffn_output), dim=1)
        #print(f"DEBUG: combined_features - Min:{combined_features.min():.2e}, Max:{combined_features.max():.2e}, Mean:{combined_features.mean():.2e}, Has_NaN:{torch.isnan(combined_features).any()}")
         
        # 4. Pass the processed hidden state through the fully connected layer
        output = self.fc(combined_features)
        #print(f"DEBUG: Final Logits - Min:{output.min():.2e}, Max:{output.max():.2e}, Mean:{output.mean():.2e}, Has_NaN:{torch.isnan(output).any()}")

        
        return output

### Initiate the model

In [10]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA available: True
CUDA device count: 4


In [11]:
class_weights = torch.tensor([weight_bonafide, weight_spoof], dtype=torch.float32).to(DEVICE)

In [12]:
model = SpoofClassifier(lstm_input_dim=config.lstm_input_dim, lstm_hidden_dim=config.lstm_hidden_dim, lstm_n_layers=config.lstm_n_layers,
                 bidirectional=config.bidirectional, dropout=config.dropout_rate, 
                 ffn_input_dim=config.ffn_input_dim, ffn_hidden_dim=config.ffn_hidden_dim,
                 output_dim=2).to(DEVICE)
criterion = torch.nn.CrossEntropyLoss(reduction='mean', weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

# print(f"DEBUG: Initial FFN_Linear WEIGHTS:\n{model.ffn_linear.weight.detach().cpu().numpy()}")
# print(f"DEBUG: Initial FFN_Linear BIAS:\n{model.ffn_linear.bias.detach().cpu().numpy()}")

### Evaluation

In [13]:
def evaluate_classifier(data_loader, model, criterion):

    model.eval()  # Set the model to evaluation mode (disables dropout, etc.)
    
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    scores_bonafide = []
    scores_spoof = []

    with torch.no_grad():  # Disable gradient calculations during evaluation
        for batch_sequences, batch_lengths, global_features, batch_labels in data_loader:
            
            batch_sequences = batch_sequences.to(DEVICE)
            batch_labels = batch_labels.to(DEVICE)
            global_features = global_features.to(DEVICE)

            # Forward pass: Get model outputs (logits)
            logits = model(batch_sequences, batch_lengths, global_features)
            
            # Calculate loss for the current batch
            loss = criterion(logits, batch_labels)
            total_loss += loss.item() * batch_sequences.size(0) # Accumulate loss, weighted by batch size

            # for EER
            probabilities = torch.softmax(logits, dim=1)
            
            for i in range(len(batch_labels)):
                current_label = batch_labels[i]
                current_score = probabilities[i]

                if current_label == LABEL_BONAFIDE:
                    scores_bonafide.append(current_score[LABEL_BONAFIDE].cpu())     # numpy is cpu only, need to move tensor from gpu
                elif current_label == LABEL_SPOOF:
                    scores_spoof.append(current_score[LABEL_BONAFIDE].cpu())
            
            total_samples += batch_labels.size(0) # Count number of samples in this batch

    average_loss = total_loss / total_samples if total_samples > 0 else 0.0

    scores_bonafide_np = np.array(scores_bonafide)    
    scores_spoof_np = np.array(scores_spoof)
    eer, threshold = em.compute_eer(scores_bonafide_np, scores_spoof_np)

    # NOTE: TBD - compute the confusion matrix
    all_scores = np.concatenate((scores_bonafide_np, scores_spoof_np))
    labels_true = np.concatenate((np.ones_like(scores_bonafide_np), np.zeros_like(scores_spoof_np)))
    labels_pred = (all_scores >= threshold).astype(int)
    
    return average_loss, eer, threshold, labels_true, labels_pred

### The training loop

>note: in wandb, scalers logs for every epoch, plots get overwritten (but still saved in artifacts?)

In [14]:
def train_model(model, train_dataloader, dev_dataloader, criterion, optimizer, num_epochs, device,
                min_eer, best_model_filename):

    print(f"Training started on device: {device}")
    model.to(device) # Ensure model is on the correct device

    # Initial metric dictionary for the progress bar
    metric_dict = {'train_loss': 'N/A', 'val_loss': 'N/A', 'val_eer': 'N/A', 'val_threshold': 'N/A'}

    # Evaluate on validation set first to get a baseline
    print("Evaluating on validation set before training...")
    model.eval() # Set model to evaluation mode
    val_loss_initial, val_eer_initial, threshold_initial, labels_true, labels_pred = evaluate_classifier(dev_dataloader, model, criterion)
    metric_dict.update({'val_loss': f'{val_loss_initial:.3f}', 'val_eer': f'{val_eer_initial*100:.2f}%', 'val_threshold': f'{threshold_initial*100:.2f}%'})
    print(f"Initial Validation - Loss: {val_loss_initial:.4f}, EER: {val_eer_initial*100:.2f}%, Threshold: {threshold_initial*100:.2f}%")

    # Progress bar setup
    total_steps = num_epochs * len(train_dataloader)
    pbar = tqdm(total=total_steps, initial=0, postfix=metric_dict, unit="batch")

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode (enables dropout, etc.)
        pbar.set_description(f"Epoch {epoch + 1}/{num_epochs}")
        
        running_train_loss = 0.0
        num_train_batches = 0

        for batch_sequences, batch_lengths, global_features, batch_labels in train_dataloader:
            # Move data to the specified device
            batch_sequences = batch_sequences.to(device)
            # batch_lengths are used by pack_padded_sequence which expects them on CPU
            batch_labels = batch_labels.to(device)
            global_features = global_features.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # # NOTE: for debug
            # # Access the weights and bias of the FFN's linear layer
            # linear_layer_weights = model.ffn_linear.weight
            # linear_layer_bias = model.ffn_linear.bias

            # print(f"DEBUG: FFN_Linear Layer WEIGHTS - Min:{linear_layer_weights.min():.2e}, Max:{linear_layer_weights.max():.2e}, Mean:{linear_layer_weights.mean():.2e}, Std:{linear_layer_weights.std():.2e}, Has_NaN:{torch.isnan(linear_layer_weights).any()}")
            # if linear_layer_bias is not None:
            #     print(f"DEBUG: FFN_Linear Layer BIAS - Min:{linear_layer_bias.min():.2e}, Max:{linear_layer_bias.max():.2e}, Mean:{linear_layer_bias.mean():.2e}, Std:{linear_layer_bias.std():.2e}, Has_NaN:{torch.isnan(linear_layer_bias).any()}")


            # Forward pass: Get model outputs (logits)
            logits = model(batch_sequences, batch_lengths, global_features)
            
            # Calculate loss
            loss = criterion(logits, batch_labels)
            
            # Backward pass and optimize
            loss.backward()
            # --- ADD THIS LINE FOR GRADIENT CLIPPING ---
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.01) # Start with max_norm=1.0 or 0.5

            optimizer.step()

            # Update statistics for progress bar and logging
            running_train_loss += loss.item()
            num_train_batches += 1
            
            pbar.update(1) # Increment progress bar by one batch
            metric_dict.update({'train_loss': f'{loss.item():.3f}'}) # Current batch loss
            pbar.set_postfix(metric_dict)
        
        # Calculate average training loss for the epoch
        avg_epoch_train_loss = running_train_loss / num_train_batches if num_train_batches > 0 else 0.0
        metric_dict.update({'train_loss': f'{avg_epoch_train_loss:.3f}'}) # Average epoch loss
        
        # Evaluate on validation set after each epoch
        avg_val_loss, val_eer, val_threshold, labels_true, labels_pred = evaluate_classifier(dev_dataloader, model, criterion)
        
        # Update with latest validation metrics
        metric_dict.update({'val_loss': f'{avg_val_loss:.3f}', 'val_eer': f'{val_eer*100:.2f}%', 'val_threshold': f'{val_threshold*100:.2f}%'})
        pbar.set_postfix(metric_dict)
        
        # Optional: Print epoch summary
        print(f"\nEpoch {epoch+1} Summary: Avg Train Loss: {avg_epoch_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, EER: {val_eer*100:.2f}%, Threshold: {val_threshold*100:.2f}%")

        # log the train\dev loss and the eer & threshold
        run.log({"train_loss": avg_epoch_train_loss, "dev_loss": avg_val_loss, 
                   "dev_eer": val_eer, "dev_threshold":val_threshold, "epoch": epoch + 1})
        
        # update min eer and optimal model
        if val_eer < min_eer:
            min_eer = val_eer
            torch.save(model.state_dict(), best_model_filename)
            print(f"Epoch {epoch+1}: New best model saved to '{best_model_filename}' with EER: {min_eer:.4f}")

            run.summary['best_validation_eer'] = min_eer
            run.summary['best_eer_epoch'] = epoch + 1
            run.summary['validation_loss_at_best_eer'] = avg_val_loss

            # log the report and confusion matrix
            class_names = ['SPOOF', 'BONAFIDE']     #NOTE: the order matters, need to match labels
            report_columns =  ["Class", "Precision", "Recall", "F1-score", "Support"]
            class_report = classification_report(labels_true, labels_pred, labels=[0, 1],
                                        target_names=class_names).splitlines()
            report_table = []
            for line in class_report[2:(len(class_names)+2)]:
                report_table.append(line.split())
            run.log({"Confusion Matix": wandb.plot.confusion_matrix(y_true=labels_true, preds=labels_pred, class_names=class_names),
                    "Classification Report": wandb.Table(data=report_table, columns=report_columns)})

    pbar.close()
    print("Training finished.")
    return min_eer

In [15]:
NUM_EPOCHS = config.epochs
min_eer = float('inf')
best_model_filename = 'best_model'  #tbc

min_eer = train_model(model, train_dataloader, dev_dataloader, criterion, optimizer, NUM_EPOCHS, DEVICE,
            min_eer, best_model_filename)

Training started on device: cuda
Evaluating on validation set before training...
Initial Validation - Loss: 0.7318, EER: 63.30%, Threshold: 52.23%


  0%|          | 0/1425 [00:00<?, ?batch/s, train_loss=N/A, val_eer=63.30%, val_loss=0.732, val_threshold=52.2…


Epoch 1 Summary: Avg Train Loss: 0.4686, Val Loss: 0.3641, EER: 19.15%, Threshold: 43.06%
Epoch 1: New best model saved to 'best_model' with EER: 0.1915
Training finished.


### Save the model

In [16]:
if min_eer != float('inf'):
    print(f"Logging the best model ({best_model_filename}) to W&B Artifacts...")
    best_model_artifact = wandb.Artifact(
        name=f"{run.id}-best-model", # Using run ID for uniqueness
        type="model",
        description=f"Best model according to EER ({min_eer:.4f}) achieved at epoch {run.summary.get('best_eer_epoch', 'N/A')}.",
        metadata={"best_eer": min_eer, "epoch_of_best_eer": run.summary.get('best_eer_epoch', 'N/A')}
    )
    best_model_artifact.add_file(best_model_filename) # Add the saved file
    wandb.run.log_artifact(best_model_artifact, aliases=["best_eer_model"]) # Add an alias
    print("Best model logged as W&B Artifact.")
else:
    print("No model was saved as best_eer did not improve from its initial value.")

run.finish()

print("W&B run finished.")

Logging the best model (best_model) to W&B Artifacts...
Best model logged as W&B Artifact.


dev_eer,▁
dev_loss,▁
dev_threshold,▁
epoch,▁
train_loss,▁
best_eer_epoch,1
best_validation_eer,0.19152
dev_eer,0.19152
dev_loss,0.36405
dev_threshold,0.43059
epoch,1


W&B run finished.
